In [125]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

In [126]:
x = np.array([[1,9,1],[2,-9,2],[-10,3,3],[4,4.01,4]]).astype(np.float32)

In [127]:
x = np.array([[1,1,1],[2,2,2],[3,3,3],[4,4,4]]).astype(np.float32)

## Perform PCA using autoencoder

In [128]:
tf.reset_default_graph()
tf.enable_eager_execution()
tf.keras.backend.clear_session()

In [129]:
class PCALayer(keras.layers.Layer):

    def __init__(self, pca_dim, **kwargs):
        self.pca_dim = pca_dim
        super(PCALayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.W = self.add_weight(
            name='W', 
            shape=(input_shape.as_list()[1], self.pca_dim),
            initializer='uniform',
            dtype='float32',
            trainable=True)
        super(PCALayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        x_proj = keras.backend.dot(x, self.W)
        x_rec = keras.backend.dot(x_proj, keras.backend.transpose(self.W))
        return x_rec

    def compute_output_shape(self, input_shape):
        return input_shape

In [130]:
epochs = 10000
batch_size = 32

In [131]:
optimizer = tf.keras.optimizers.Adamax()
global_step = tf.Variable(0)
pca_layer = PCALayer(1)
inp=keras.layers.Input(shape=(x.shape[1],))
model = keras.models.Model(inputs=inp, outputs=pca_layer(inp))

In [132]:
def loss(model,x,y):
    y_ = model(x)
    return tf.losses.mean_squared_error(y,y_)

In [133]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [134]:
#model.get_weights()

In [136]:
for epoch in range(epochs):
    optimizer.minimize(loss(model,x,x), model.trainable_variables)
    print(loss(model,x,x))

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object is not callable